### [FW] Plot wind rose for different methods

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# PLOT WITH SNS
import seaborn as sns
sns.set_style("whitegrid")

env_layouts = ["4Symm", "8LHS", "16LHS"]
layout_prefix = ["4Symm_", "8LHS_", "16LHS_"]
methods = ["FW_PropSR", "FW_Naive", "FW_Random", "FW_PPO"]
methods_display_names = ["PropSR", "Naive", "Random", "PPO"]

for layout, prefix in zip(env_layouts, layout_prefix):

    # plot wind rose for each method
    powers = {}
    for method in methods:
        try: powers[f"{method}"] = np.load(f"data/eval/scores/fw_wind_rose/{layout}_{method}_total_powers.npy")[0]
        except: 
            powers[f"{method}"] = np.zeros_like(powers[f"{methods[0]}"]) + np.min(powers[f"{methods[0]}"])
            print(f"Could not load {layout}_{method}_total_powers.npy")

    for method, methods_display_name in zip(methods, methods_display_names):
        total_powers = powers[f"{method}"]
        mean_total_powers = np.mean(total_powers, axis=1)
        angles = np.linspace(0, 360, mean_total_powers.shape[0])
        sns.lineplot(x=angles, y=mean_total_powers, label=methods_display_name)
    plt.title(f"Generated power for different wind directions ({layout})")
    plt.xlabel("Wind direction")
    plt.ylabel("Mean total power")
    plt.legend()
    plt.savefig(f"data/eval/plots/fw_wind_rose_{layout}.png")
    plt.show()

    for method, methods_display_name in zip(methods, methods_display_names):
        if method in ["FW_Naive", "FW_Random"]: continue
        total_powers = powers[f"{method}"]
        mean_total_powers = np.mean(total_powers, axis=1) - np.mean(powers["FW_Naive"], axis=1)
        angles = np.linspace(0, 360, mean_total_powers.shape[0])
        sns.lineplot(x=angles, y=mean_total_powers, label=methods_display_name)
    plt.title(f"Generated power for different wind directions \nwrt Naive policy ({layout})", wrap=True)
    plt.xlabel("Wind direction")
    plt.ylabel("Mean total power increase")
    plt.ticklabel_format(axis='both', style='sci', scilimits=(0,0))
    plt.legend()
    plt.savefig(f"data/eval/plots/fw_wind_rose_{layout}_wrt_naive.png")
    plt.show()

    # plot final power instead
    for method, methods_display_name in zip(methods, methods_display_names):
        total_powers = powers[f"{method}"]
        final_total_powers = total_powers[:, -1]
        angles = np.linspace(0, 360, final_total_powers.shape[0])
        plt.plot(angles, final_total_powers, label=methods_display_name)
    plt.title(f"Generated power for different wind directions ({layout})")
    plt.xlabel("Wind direction")
    plt.ylabel("Mean last step power")
    plt.legend()
    plt.savefig(f"data/eval/plots/fw_wind_rose_final_step_{layout}.png")
    plt.show()

In [ ]:
# Plot results
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
plt.ioff()


# Model improvements and yearly improvements
improvements_df = pd.DataFrame(columns=["Env_layout", "Env_type", "Method", "Power", "Power increase", "Yearly energy increase"])
for env_layout in ["4Symm", "8LHS", "16LHS"]:
    # Fixed wind results
    fw_propsr_power = np.load(f"data/eval/scores/{env_layout}_FW_PropSR_total_powers.npy").sum(axis=2).mean(axis=1) # Mean over the 10 eval seeds
    fw_naive_power = np.load(f"data/eval/scores/{env_layout}_FW_Naive_total_powers.npy").sum(axis=2).mean(axis=1)
    fw_random_power = np.load(f"data/eval/scores/{env_layout}_FW_Random_total_powers.npy").sum(axis=2).mean(axis=1)
    fw_ppo_power = np.load(f"data/eval/scores/{env_layout}_FW_PPO_total_powers.npy")
    fw_results = {
        "FlorisSR": fw_propsr_power, # [1seed]
        "Naive": fw_naive_power, # [1seed]
        "Random": fw_random_power, # [10seeds]
        "PPO": fw_ppo_power, # [2seeds]
    }

    # Changing wind results
    cw_propsr_power = np.load(f"data/eval/scores/{env_layout}_CW_PropSR_total_powers.npy").sum(axis=2).mean(axis=1) # Mean over the 10 eval seeds, sum over the 100 steps
    cw_naive_power = np.load(f"data/eval/scores/{env_layout}_CW_Naive_total_powers.npy").sum(axis=2).mean(axis=1) 
    cw_random_power = np.load(f"data/eval/scores/{env_layout}_CW_Random_total_powers.npy").sum(axis=2).mean(axis=1) 
    cw_ppo_power = np.load(f"data/eval/scores/{env_layout}_CW_PPO_total_powers.npy")
    cw_results = {
        "FlorisSR": cw_propsr_power,
        "Naive": cw_naive_power,
        "Random": cw_random_power,
        "PPO": cw_ppo_power, # its already summed for ppo
    }

    # 16LHS is not used for dyn experiments
    if env_layout != "16LHS":
        # Dynamic results
        dyn_propsr_power = np.load(f"data/eval/scores/{env_layout}_DynOP_PropSR_total_powers.npy").sum(axis=2).mean(axis=1)
        dyn_naive_power = np.load(f"data/eval/scores/{env_layout}_DynOP_Naive_total_powers.npy").sum(axis=2).mean(axis=1)
        dyn_random_power = np.load(f"data/eval/scores/{env_layout}_DynOP_Random_total_powers.npy").sum(axis=2).mean(axis=1)
        dyn_ppo_power =np.load(f"data/eval/scores/{env_layout}_DynOP_PPO_total_powers.npy")
        dyn_ppo_p0_power = np.load(f"data/eval/scores/{env_layout}_DynOP_p0_PPO_total_powers.npy")
        dyn_ppo_p1_power = np.load(f"data/eval/scores/{env_layout}_DynOP_p1_PPO_total_powers.npy")
        dyn_ppo_p2_power = np.load(f"data/eval/scores/{env_layout}_DynOP_p2_PPO_total_powers.npy")
        id = np.argmax([dyn_ppo_p0_power.mean(), dyn_ppo_p1_power.mean(), dyn_ppo_p2_power.mean()])
        print(f"{env_layout} | Dynamic PPO p{id} power: {dyn_ppo_p0_power.mean()} {dyn_ppo_p1_power.mean()} {dyn_ppo_p2_power.mean()}")	
        dyn_ppo_priv_power = [dyn_ppo_p0_power, dyn_ppo_p1_power, dyn_ppo_p2_power][id]
        dyn_results = {
            "FlorisSR": dyn_propsr_power,
            "Naive": dyn_naive_power,
            "Random": dyn_random_power,
            "PPO": dyn_ppo_power,
            "PPO priv.": dyn_ppo_priv_power,
        }

    values = [
        (fw_results.values(), fw_results.keys(), "FLORIS Fixed wind environment", "FW"), 
        (cw_results.values(), cw_results.keys(), "FLORIS Changing wind environment", "CW"),]
    if env_layout != "16LHS": 
        values += [(dyn_results.values(), dyn_results.keys(), "FloriDyn", "DynOP")]
    
    # Plotting a boxplot, a barplot and printing the results
    for data, names, title, save_prefix in values:
        
        MAX_EVAL_REPS = 10
        data = [d[:10] for d in data]
        #print(names, data)

        # get data of naive
        # naive = data[1]
        _names = list(names)
        print(title, env_layout)
        print(f"Naive: {np.mean(data[1])} ({np.std(data[1])}) ({_names[1]})")
        naive = np.mean(data[1])
        print("Improvement over naive:")
        for i in range(len(data)):
            name = _names[i]
            d = data[i]
            print(f"\t{name} {np.mean(d) - naive}")
        print("Yearly improvement - percentage improvement:")
        for i in range(len(data)):
            name = _names[i]
            d = data[i]
            imp = np.mean(d) - naive
            imp_power = imp / 100 # Watts [Actually, isnt this in MWh?]
            wattyear = imp_power * 8760 / 1e6 # in MWh
            print(f"\t{name} {round(wattyear)} - ({round(imp/naive*100, 4)})")
            improvements_df.loc[len(improvements_df)] = [
                env_layout, save_prefix, name, np.mean(d)/100, imp_power, wattyear]
        
        # Mean power barplot
        # long figure
        plt.figure()
        # y = [np.mean(d, axis=0)/1e6/100 for d in data]
        y = [d / 1e6 / 100 for d in data]
        # for d in data:
        #     print(d.shape)
        
        # plt.ylim(np.min(y) - (np.max(y) - np.min(y)) * 0.1, np.max(y) + (np.max(y) - np.min(y)) * 0.1)
        # bpdata = {name: d for name, d in zip(names, data)}
        # sns.barplot(data=bpdata, hue_order=names, errorbar="ci")
        df = pd.DataFrame(columns=["Method", "Power"])
        for i, d in enumerate(data):
            for j, val in enumerate(d):
                # if "PPO" in _names[i]:
                #     # print("aAAAAA", val.shape)
                #     val = val[0][0]
                df.loc[len(df)] = [_names[i], val]
        df["Power"] = df["Power"] / 1e6 / 100
        #print(df)
        sns.barplot(data=df, x="Method", y="Power", hue="Method",
                    hue_order=["FlorisSR", "Naive", "Random", "PPO", "PPO priv.", "PPO p0", "PPO p1", "PPO p2", "recPPO"],
                    errorbar="sd", err_kws={"alpha": 0.75}, capsize=0.1)
        # plt.bar(names, [np.mean(d) for d in data])

        y = [np.mean(_y) + np.std(_y) for _y in y] + [np.mean(_y) - np.std(_y) for _y in y]
        h = np.max(y) - np.min(y)
        plt.ylim(np.min(y) - (np.max(y) - np.min(y)) * 0.1, np.max(y) + (np.max(y) - np.min(y)) * 0.1)

        for i, d in enumerate(data):
            if np.mean(d) == 0: plt.text(i, 0, "N/A", ha="center")
            else: 
                _y = d / 1e6 / 100
                _y = np.mean(_y) #+ np.std(_y)
                plt.text(i, _y, f"{round(np.mean(d)/1e6/100, 2):.02f} MW", ha="center")#, bbox=dict(facecolor='white', edgecolor='black', alpha=1))#, pad=1))
        plt.title(f"{title} average generated power ({env_layout})")
        plt.xlabel("Method")
        plt.ylabel("Mean total power [MW]")
        plt.xticks(rotation=20)
        plt.savefig(f"data/eval/plots/{save_prefix}_{env_layout}_barplot.png", bbox_inches="tight")
        # plt.show()

        # Barplot wrt the random method
        if not "Dyn" in title: 
            plt.figure()
            datadict = {name: d for name, d in zip(names, data)}
            random_avg = np.mean(datadict["Random"])
            datadict = {name: d - random_avg for name, d in datadict.items()}
            datadict = {name: d for name, d in datadict.items() if not "Random" in name}
            x = list(datadict.keys())
            y = [np.mean(d) for d in datadict.values()]
            sns.barplot(x=x, y=y, hue=x, hue_order=["FlorisSR", "Naive", "Random", "PPO", "PPO priv.", "PPO p0", "PPO p1", "PPO p2", "recPPO"])
            for i, d in enumerate(datadict.values()):
                percentage_impact = np.mean(d) / random_avg * 100
                sign = "+" if percentage_impact > 0 else ""
                y = np.mean(d) if np.mean(d) > 0 else 1.1*np.mean(d)
                plt.text(i, y, f"{round(np.mean(d/1e6 / 100), 2):.02f} MW ({sign}{percentage_impact:.02f}%)", ha="center")
            plt.title(f"{title} average power increase \nover random policy ({env_layout})", wrap=True)
            plt.xticks(rotation=20)
            plt.savefig(f"data/eval/plots/{save_prefix}_{env_layout}_barplot wrt Random.png")
            # plt.show()

        # Barplot wrt naive
        plt.figure()
        datadict = {name: d for name, d in zip(names, data)}
        naive_avg = np.mean(datadict["Naive"])
        datadict = {name: d - naive_avg for name, d in datadict.items()}
        datadict = {name: d for name, d in datadict.items() if not "Naive" in name}
        x = list(datadict.keys())
        y = [np.mean(d) for d in datadict.values()]
        sns.barplot(x=x, y=y, hue=x, hue_order=["FlorisSR", "Naive", "Random", "PPO", "PPO priv.", "PPO p0", "PPO p1", "PPO p2", "recPPO"])
        for i, d in enumerate(datadict.values()):
            percentage_impact = np.mean(d) / naive_avg * 100
            sign = "+" if percentage_impact > 0 else ""
            #if np.mean(d) == 0: plt.text(i, 0, "N/A", ha="center")
            #else:  
            y = np.mean(d) * 1.1 if np.mean(d) > 0 else np.mean(d)* 0.9
            plt.text(i, y, f"{round(np.mean(d/1e6 / 100), 2):.02f} MW ({sign}{percentage_impact:.02f}%)", ha="center")
        plt.title(f"{title} average power change \nover naive policy ({env_layout})", wrap=True)
        plt.xticks(rotation=20)
        plt.savefig(f"data/eval/plots/{save_prefix}_{env_layout}_barplot wrt Naive.png")
        # plt.show()

        # Print results 
        # print(title)
        # for name, d in zip(names, data):
        #     print(name, np.mean(d))
        print("")

### Plot results side by side


In [ ]:
# Plots side by side
for env_type in ["FW", "CW", "DynOP"]:
    # plot with multiple figures
    fig = plt.figure()
    fig.set_size_inches(18, 4)
    layouts = ["4Symm", "8LHS", "16LHS"] if env_type != "DynOP" else ["4Symm", "8LHS"]
    for i, env_layout in enumerate(layouts):
        ax = fig.add_subplot(1, 3, i+1)
        img = plt.imread(f"data/eval/plots/{env_type}_{env_layout}_barplot.png")
        ax.imshow(img)
        plt.axis("off")
    plt.show()